In [3]:
# Load modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from biogeme.expressions import Beta, Variable, log
from biogeme.models import loglogit

import biogeme.biogeme as bio
import biogeme.database as db

In [4]:
# Load data
data_train = pd.read_csv('data/swissmetro_clean_train.csv')
data_test = pd.read_csv('data/swissmetro_clean_test.csv')

J = 3
K = 2
Xvars = ['TRAIN_COST','TRAIN_TT','TRAIN_HE','SM_COST','SM_TT','SM_HE','CAR_COST','CAR_TT']
X_test = data_test[Xvars].to_numpy()

# Set database
database_train = db.Database('swissmetro_train',data_train)
database_test = db.Database('swissmetro_test',data_test)

# Set variables
CHOICE = Variable('CHOICE')

TRAIN_TT = Variable('TRAIN_TT')
TRAIN_COST = Variable('TRAIN_COST')
TRAIN_HE = Variable('TRAIN_HE')
SM_TT = Variable('SM_TT')
SM_COST = Variable('SM_COST')
SM_HE = Variable('SM_HE')
CAR_TT = Variable('CAR_TT')
CAR_COST = Variable('CAR_COST')

In [5]:
# Set betas
B_TRAIN_TT = Beta('B_TRAIN_TT',0,None,None,0)
B_TRAIN_HE = Beta('B_TRAIN_HE',0,None,None,0)
B_SM_TT = Beta('B_SM_TT',0,None,None,0)
B_SM_HE = Beta('B_SM_HE',0,None,None,0)
B_CAR_TT = Beta('B_CAR_TT',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)
ASC_SM = Beta('ASC_SM',0,None,None,0)
ASC_CAR = Beta('ASC_CAR',0,None,None,0)

# Set utility functions
V1 = TRAIN_TT * B_TRAIN_TT + TRAIN_COST * B_COST + TRAIN_HE * B_TRAIN_HE
V2 = ASC_SM + SM_TT * B_SM_TT + SM_COST * B_COST + SM_HE * B_SM_HE
V3 = ASC_CAR + CAR_TT * B_CAR_TT + CAR_COST * B_COST

V = {1: V1, 2: V2, 3: V3}

# Set availability conditions
av = {1: 1, 2: 1, 3: 1}

# Set model as MNL
logprob = loglogit(V,av,CHOICE)


# Set biogeme object to estimate
model = bio.BIOGEME(database_train,logprob)

# Set model to silent output
model.generateHtml = False
model.generatePickle = False
model.saveIterations = False
model.modelName = None

# Estimate
results = model.estimate()

In [6]:
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,-0.989797,0.165520,-5.979939,2.232206e-09
ASC_SM,-0.211486,0.169460,-1.248005,2.120292e-01
B_CAR_TT,-1.004023,0.070591,-14.223017,0.000000e+00
B_COST,-0.811762,0.055282,-14.684051,0.000000e+00
B_SM_HE,-0.730202,0.310567,-2.351192,1.871335e-02
B_SM_TT,-1.507886,0.115079,-13.103031,0.000000e+00
B_TRAIN_HE,-0.776713,0.123892,-6.269263,3.627609e-10
B_TRAIN_TT,-2.044770,0.113905,-17.951606,0.000000e+00


In [7]:
# Get log-likelihood in train and test sample
model_test = bio.BIOGEME(database_test,logprob)

ll_train = model.simulate(theBetaValues=results.getBetaValues()).to_numpy().sum()
ll_test = model_test.simulate(theBetaValues=results.getBetaValues()).to_numpy().sum()
ll_full = ll_train + ll_test
r2_test = 1 - ll_test/(len(X_test)*np.log(1/J))

# Create metrics dataframe
metrics = pd.Series(np.r_[ll_full,ll_train,ll_test,r2_test],index=['Log-likelihood (full)','Log-likelihood (train)','Log-likelihood (test)','Rho-squared (test)'],name='Value')
# metrics.to_csv('results/mnl_linear_swissmetro_metrics.csv')
metrics

Log-likelihood (full)    -7214.938388
Log-likelihood (train)   -5767.311925
Log-likelihood (test)    -1447.626463
Rho-squared (test)           0.271191
Name: Value, dtype: float64

In [8]:
# MU and VTT
pars = results.getEstimatedParameters()

# Get marginal utilities
mu_train_cost = pars['Value']['B_COST']
mu_train_tt   = pars['Value']['B_TRAIN_TT']
mu_train_he   = pars['Value']['B_TRAIN_HE']
mu_sm_cost    = pars['Value']['B_COST']
mu_sm_tt      = pars['Value']['B_SM_TT']
mu_sm_he      = pars['Value']['B_SM_HE']
mu_car_cost   = pars['Value']['B_COST']
mu_car_tt     = pars['Value']['B_CAR_TT']

mu_array = np.array([mu_train_cost,mu_train_tt,mu_train_he,mu_sm_cost,mu_sm_tt,mu_sm_he,mu_car_cost,mu_car_tt])


In [9]:
df_mu = pd.Series(mu_array,index=Xvars,name='Value')
# df_mu.to_csv('results/mnl_linear_swissmetro_mu.csv')
df_mu

TRAIN_COST   -0.811762
TRAIN_TT     -2.044770
TRAIN_HE     -0.776713
SM_COST      -0.811762
SM_TT        -1.507886
SM_HE        -0.730202
CAR_COST     -0.811762
CAR_TT       -1.004023
Name: Value, dtype: float64

In [10]:
vtt_train      = mu_train_tt/mu_train_cost
vtt_sm         = mu_sm_tt/mu_sm_cost
vtt_car        = mu_car_tt/mu_car_cost

vtt_array = np.array([vtt_train,vtt_sm,vtt_car])

In [11]:
vtt_names = ['TRAIN', 'SM', 'CAR']
df_vtt = pd.Series(vtt_array,index=vtt_names,name='Value')
# df_vtt.to_csv('results/mnl_linear_swissmetro_vtt.csv')
df_vtt

TRAIN    2.518927
SM       1.857547
CAR      1.236844
Name: Value, dtype: float64

In [12]:
vowt_train      = mu_train_he/mu_train_cost
vowt_sm         = mu_sm_he/mu_sm_cost

vowt_array = np.array([vowt_train,vowt_sm])

In [13]:
vowt_names = ['TRAIN', 'SM']
df_vowt = pd.Series(vowt_array,index=vowt_names,name='Value')
# df_vowt.to_csv('results/mnl_linear_swissmetro_vowt.csv')
df_vowt

TRAIN    0.956823
SM       0.899527
Name: Value, dtype: float64

In [14]:
# Set betas
B_TRAIN_TT = Beta('B_TRAIN_TT',0,None,None,0)
B_TRAIN_HE = Beta('B_TRAIN_HE',0,None,None,0)
B_CAR_TT = Beta('B_CAR_TT',0,None,None,0)
B_SM_TT = Beta('B_SM_TT',0,None,None,0)
B_SM_HE = Beta('B_SM_HE',0,None,None,0)
B_CAR_TT = Beta('B_CAR_TT',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)
ASC_SM = Beta('ASC_SM',0,None,None,0)
ASC_CAR = Beta('ASC_CAR',0,None,None,0)

# Set utility functions
V1 = log(TRAIN_TT + 0.1) * B_TRAIN_TT + log(TRAIN_COST + 0.1) * B_COST + log(TRAIN_HE + 0.1) * B_TRAIN_HE
V2 = ASC_SM + log(SM_TT + 0.1) * B_SM_TT + log(SM_COST + 0.1) * B_COST + log(SM_HE + 0.1) * B_SM_HE
V3 = ASC_CAR + log(CAR_TT + 0.1) * B_CAR_TT + log(CAR_COST + 0.1) * B_COST

V = {1: V1, 2: V2, 3: V3}

# Set availability conditions
av = {1: 1, 2: 1, 3: 1}

# Set model as MNL
logprob = loglogit(V,av,CHOICE)


# Set biogeme object to estimate
model = bio.BIOGEME(database_train,logprob)

# Set model to silent output
model.generateHtml = False
model.generatePickle = False
model.saveIterations = False
model.modelName = None

# Estimate
results = model.estimate()

In [15]:
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,0.609380,0.072142,8.446999,0.000000e+00
ASC_SM,0.256948,0.135788,1.892274,5.845450e-02
B_CAR_TT,-1.674178,0.080758,-20.730855,0.000000e+00
B_COST,-1.058442,0.052412,-20.194721,0.000000e+00
B_SM_HE,-0.195157,0.091880,-2.124045,3.366640e-02
B_SM_TT,-1.949839,0.083282,-23.412409,0.000000e+00
B_TRAIN_HE,-0.598454,0.092806,-6.448471,1.129838e-10
B_TRAIN_TT,-3.548927,0.131488,-26.990514,0.000000e+00


In [16]:
# Get log-likelihood in train and test sample
model_test = bio.BIOGEME(database_test,logprob)

ll_train = model.simulate(theBetaValues=results.getBetaValues()).to_numpy().sum()
ll_test = model_test.simulate(theBetaValues=results.getBetaValues()).to_numpy().sum()
ll_full = ll_train + ll_test
r2_test = 1 - ll_test/(len(X_test)*np.log(1/J))

# Create metrics dataframe
metrics = pd.Series(np.r_[ll_full,ll_train,ll_test,r2_test],index=['Log-likelihood (full)','Log-likelihood (train)','Log-likelihood (test)','Rho-squared (test)'],name='Value')
# metrics.to_csv('results/mnl_loglinear_swissmetro_metrics.csv')
metrics

Log-likelihood (full)    -6994.602442
Log-likelihood (train)   -5591.367118
Log-likelihood (test)    -1403.235324
Rho-squared (test)           0.293540
Name: Value, dtype: float64

In [17]:
# MU and VTT
pars = results.getEstimatedParameters()

mu_train_cost = pars['Value']['B_COST']/(X_test[:,0] + 0.1)
mu_train_tt   = pars['Value']['B_TRAIN_TT']/(X_test[:,1] + 0.1)
mu_train_he   = pars['Value']['B_TRAIN_HE']/(X_test[:,2] + 0.1)
mu_sm_cost    = pars['Value']['B_COST']/(X_test[:,3] + 0.1)
mu_sm_tt      = pars['Value']['B_SM_TT']/(X_test[:,4] + 0.1)
mu_sm_he      = pars['Value']['B_SM_HE']/(X_test[:,5] + 0.1)
mu_car_cost   = pars['Value']['B_COST']/(X_test[:,6] + 0.1)
mu_car_tt     = pars['Value']['B_CAR_TT']/(X_test[:,7] + 0.1)

mu_array = np.c_[mu_train_cost,mu_train_tt,mu_train_he,mu_sm_cost,mu_sm_tt,mu_sm_he,mu_car_cost,mu_car_tt]

# Create statistics
mu_mean   = np.mean(mu_array,axis=0)
mu_std    = np.std(mu_array,axis=0)
mu_median = np.median(mu_array,axis=0)

In [18]:
# Get percentiles of VTT
mu_perc=np.quantile(mu_array,q=[0,.05,.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,1],axis=0)
pd.DataFrame(mu_perc,columns=Xvars)

,TRAIN_COST,TRAIN_TT,TRAIN_HE,SM_COST,SM_TT,SM_HE,CAR_COST,CAR_TT
0,-10.584417,-6.958681,-1.496136,-10.584417,-6.963710,-0.975783,-5.880232,-3.986137
1,-10.584417,-4.207516,-1.496136,-10.584417,-4.431452,-0.975783,-2.520099,-2.536633
2,-3.564637,-3.584775,-1.496136,-3.024119,-3.899677,-0.975783,-2.116883,-2.092722
3,-2.035465,-3.007565,-1.496136,-1.764070,-3.094982,-0.975783,-1.707164,-1.708345
4,-1.628372,-2.609505,-1.496136,-1.411256,-2.671012,-0.975783,-1.411256,-1.468577
5,-1.356977,-2.246156,-0.854935,-1.138109,-2.349203,-0.650522,-1.230746,-1.287829
6,-1.163123,-1.982641,-0.854935,-0.989198,-2.096601,-0.650522,-1.126002,-1.146697
7,-1.008040,-1.810677,-0.854935,-0.867575,-1.856989,-0.650522,-0.962220,-1.008541
8,-0.882035,-1.650664,-0.460350,-0.740169,-1.652406,-0.487892,-0.867575,-0.900096
9,-0.735029,-1.493663,-0.460350,-0.604824,-1.481661,-0.487892,-0.756030,-0.767971


In [19]:
df_mu = pd.DataFrame(np.c_[mu_mean,mu_median,mu_std],index=Xvars,columns=['Mean','Median','SD'])
# df_mu.to_csv('results/mnl_loglinear_swissmetro_mu.csv')
df_mu

,Mean,Median,SD
TRAIN_COST,-2.031156,-1.163123,2.615091
TRAIN_TT,-2.284460,-1.982641,1.019811
TRAIN_HE,-0.921827,-0.854935,0.426834
SM_COST,-1.851571,-0.989198,2.634651
SM_TT,-2.337633,-2.096601,1.062273
SM_HE,-0.705032,-0.650522,0.202707
CAR_COST,-1.277253,-1.126002,0.683474
CAR_TT,-1.299706,-1.146697,0.630121


In [20]:
# Get VTT
vtt_train      = mu_train_tt/mu_train_cost
vtt_sm         = mu_sm_tt/mu_sm_cost
vtt_car        = mu_car_tt/mu_car_cost

vtt_array = np.c_[vtt_train,vtt_sm,vtt_car]

# Create statistics
vtt_mean   = np.mean(vtt_array,axis=0)
vtt_std    = np.std(vtt_array,axis=0)
vtt_median = np.median(vtt_array,axis=0)

vtt_names = ['TRAIN', 'SM', 'CAR']

In [21]:
# Get percentiles of VTT
vtt_perc=np.quantile(vtt_array,q=[0,.05,.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,1],axis=0)
pd.DataFrame(vtt_perc,columns=vtt_names)

,TRAIN,SM,CAR
0,0.092368,0.060999,0.018135
1,0.306791,0.319567,0.609793
2,0.699809,0.673607,0.673307
3,1.060342,1.154124,0.787445
4,1.267408,1.453014,0.869956
5,1.497445,1.694743,0.949043
6,1.713848,2.016254,1.034213
7,1.938608,2.380662,1.107217
8,2.219429,2.859532,1.201738
9,2.637456,3.430470,1.335135


In [22]:
# Drop outliers
vtt_train_clean = vtt_train[(vtt_train>=0) & (vtt_train <= vtt_perc[-2,0])]
vtt_sm_clean = vtt_sm[(vtt_sm>=0) & (vtt_sm <= vtt_perc[-2,1])]
vtt_car_clean = vtt_car[(vtt_car>=0) & (vtt_car <= vtt_perc[-2,2])]

# Create clean VTT statistic arrays
mean_vtt_array = np.r_[np.mean(vtt_train_clean),np.mean(vtt_sm_clean),np.mean(vtt_car_clean)]
median_vtt_array = np.r_[np.median(vtt_train_clean),np.median(vtt_sm_clean),np.median(vtt_car_clean)]
std_vtt_array = np.r_[np.std(vtt_train_clean),np.std(vtt_sm_clean),np.std(vtt_car_clean)]

min_vtt_array = np.r_[np.min(vtt_train_clean),np.min(vtt_sm_clean),np.min(vtt_car_clean)]
max_vtt_array = np.r_[np.max(vtt_train_clean),np.max(vtt_sm_clean),np.max(vtt_car_clean)]

In [23]:
df_vtt = pd.DataFrame(np.c_[mean_vtt_array,median_vtt_array],index=vtt_names,columns=['Mean','Median'])
# df_vtt.to_csv('results/mnl_loglinear_swissmetro_vtt.csv')
df_vtt

,Mean,Median
TRAIN,1.732687,1.652874
SM,2.133015,1.932041
CAR,1.030338,1.016313


In [24]:
# Get VoWT
vowt_train      = mu_train_he/mu_train_cost
vowt_sm         = mu_sm_he/mu_sm_cost

vowt_array = np.c_[vowt_train,vowt_sm]

# Create statistics
vowt_mean   = np.mean(vowt_array,axis=0)
vowt_std    = np.std(vowt_array,axis=0)
vowt_median = np.median(vowt_array,axis=0)

vowt_names = ['TRAIN', 'SM']

In [25]:
# Get percentiles of VTT
vowt_perc=np.quantile(vowt_array,q=[0,.05,.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,1],axis=0)
pd.DataFrame(vowt_perc,columns=vowt_names)

,TRAIN,SM
0,0.043493,0.046095
1,0.080773,0.092191
2,0.178135,0.193600
3,0.315015,0.341105
4,0.428345,0.460953
5,0.551276,0.561748
6,0.702725,0.663772
7,0.857405,0.811277
8,1.074281,0.977220
9,1.351332,1.180039


In [26]:
# Drop outliers
vowt_train_clean = vowt_train[(vowt_train>=0) & (vowt_train <= vowt_perc[-2,0])]
vowt_sm_clean = vowt_sm[(vowt_sm>=0) & (vowt_sm <= vowt_perc[-2,1])]

# Create clean VoWT statistic arrays
mean_vowt_array = np.r_[np.mean(vowt_train_clean),np.mean(vowt_sm_clean)]
median_vowt_array = np.r_[np.median(vowt_train_clean),np.median(vowt_sm_clean)]

In [27]:
df_vowt = pd.DataFrame(np.c_[mean_vowt_array,median_vowt_array],index=vowt_names,columns=['Mean','Median'])
# df_vowt.to_csv('results/mnl_loglinear_swissmetro_vowt.csv')
df_vowt

,Mean,Median
TRAIN,0.780458,0.662338
SM,0.718913,0.636115
